<a href="https://colab.research.google.com/github/selfor21/NN-FROM-SCRATCH-LOW-LEVEL/blob/main/C_05_NN_Error_with_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculating Network Error with Loss

## Libraries and Data

In [2]:
pip install nnfs

In [3]:
import numpy as np
import math
import nnfs
nnfs.init()

In [4]:
import matplotlib.pyplot as plt
from nnfs.datasets import spiral_data

In [7]:
# Create dataset
X, y = spiral_data(samples=100, classes=3)


## Classes Layer_Dense, ReLU and Softmax Activation Function

In [5]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
      self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
      self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
      self.output = np.dot(inputs, self.weights) + self.biases


In [6]:
class Activation_ReLU:
  # Forward Pass
  def forward(self, inputs):
    self.output = np.maximum(0,inputs)


In [ ]:
# Softmax activation
class Activation_Softmax:
# Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = np.exp(inputs - np.max(inputs, axis=1,
    keepdims=True))
  # Normalize them for each sample
    probabilities = exp_values / np.sum(exp_values, axis=1,
    keepdims=True)
    self.output = probabilities

## Categorical Cross-Entropy Loss